# SQLite with pandas

*SQLite* is a database engine that stores data in single files that can be easily shared. It is supported by many programming languages, such as python, that provides a library to access SQLite databases.

We see how to use such a library, called *sqlite3*, to create, update, and query databases. In particular, we see how the *pandas* package simplifies working with SQLite databases 

## Import libraries 

In [1]:
import pandas as pd
import sqlite3

In order to work with a *SQLite* database from python, we have to make a *connection* using the **connect** function which returns a *connection* object. 

Below, we specify the (path to the) database's name, *iTunes*, in our case. 
Note that if the database does not exist, the connect function creates the empty database. Otherwise, a connection would be made to the existing database.

In [2]:
# connection to a database
db = sqlite3.connect('iTunes.db') 

Then, we create a *cursor* object that allows python to execute SQL queries.

In [3]:
# create cursor to execute SQL queries
cur = db.cursor()

## A very simple music database 

The picture below shows the database that we are going to create. 

It contains $4$ tables, *track*, *album*, *genre*, and *artist*. Each of them contains a proper primary key. 
The *track* table also contains a foreign key, *genre_id*, pointing to *genre* table, and *album_id*, pointing to *album* table. The *album* table, in turn, also contains a foreign key, *artist_id*, to the *artist* table.

<img height=900 width=700 src="database.png" />

## Create tables

The first step is create the tables above.

SQL queries are executed using *execute* from the cursor object.
Moreover, the database is modified only when the modification is committed using *commit* from the database object.

In [4]:
# artist table:
# - a primary key
# - name of artist
cur.execute("""CREATE TABLE IF NOT EXISTS artist (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name TEXT
    )"""
)
db.commit()

In [5]:
# genre table:
# - a primary key
# - name of genre
cur.execute("""CREATE TABLE IF NOT EXISTS genre (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name TEXT
    )"""
)
db.commit()

In [6]:
# album table:
# - a primary key
# - a foreign key pointing to the artist table
# - name of the album
cur.execute("""CREATE TABLE IF NOT EXISTS album (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id INTEGER,
    title TEXT
    )"""
)
db.commit()

In [7]:
# track table:
# - a primary key
# - a foreign key pointing to the album table
# - a foreign key pointing to the genre table
# - track title, a rating, the length, the reproduction
cur.execute("""CREATE TABLE IF NOT EXISTS track (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    album_id INTEGER,
    genre_id INTEGER,
    title TEXT,
    len INTEGER,
    rating INTEGER,
    repr INTEGER
    )"""
)
db.commit()

### SQL queries using pandas

pandas provides the **read_sql_query** function which permits to execute SQL queries and returns pandas *DataFrames*. Of course, this has some advantages, such as automatically reading the names of the headers from the table as well as visualise data easily through a dataframe.

Let us double check that our tables have been created:

In [8]:
pd.read_sql_query("SELECT * FROM sqlite_master WHERE type='table'",db) 

,type,name,tbl_name,rootpage,sql
0,table,artist,artist,2,CREATE TABLE artist (\n id INTEGER NOT NULL...
1,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,genre,genre,5,CREATE TABLE genre (\n id INTEGER NOT NULL ...
3,table,album,album,7,CREATE TABLE album (\n id INTEGER NOT NULL ...
4,table,track,track,9,CREATE TABLE track (\n id INTEGER NOT NULL ...


Verify that tables are still empty.

In [9]:
pd.read_sql_query("SELECT * FROM track", db)

,id,album_id,genre_id,title,len,rating,repr


## Fill tables

Let us now populate the tables.

#### Populate the artist table

In [10]:
values = zip(['Led Zeppelin','Genesis','Metallica','AC/DC','Pink Floyd','The Cure','The Smiths'])
for artistname in values:
    cur.execute("INSERT INTO artist (name) VALUES (?)",artistname)
#endfor
db.commit()

In [11]:
# double check 
pd.read_sql_query("SELECT * FROM artist", db)

,id,name
0,1,Led Zeppelin
1,2,Genesis
2,3,Metallica
3,4,AC/DC
4,5,Pink Floyd
5,6,The Cure
6,7,The Smiths


#### Populate the genre table

In [12]:
values = zip(['Rock','Metal','Pop','Progressive rock','Classic','Alternative rock'])
for genre in values:
    cur.execute("INSERT INTO genre (name) VALUES (?)",genre)
#endfor
db.commit()

In [13]:
# double check 
pd.read_sql_query("SELECT * FROM genre", db)

,id,name
0,1,Rock
1,2,Metal
2,3,Pop
3,4,Progressive rock
4,5,Classic
5,6,Alternative rock


#### Populate the album table

In this table, we have to fill the foreign key column *artist_id*. The numbers here have to be equal to the corresponding primary key of the artist table. 

In [14]:
cur.execute("""INSERT INTO album (title,artist_id) VALUES 
    ('Who Made Who',4),
    ('Physical Graffiti',1),
    ('The dark side of the moon',5),
    ('Ride the Lightning',3),
    ('Selling England by the pound',2),
    ('The Queen Is Dead',7),
    ('Wish',6)
    """)
db.commit()

In [15]:
# double check 
pd.read_sql_query("SELECT * FROM album", db)

,id,artist_id,title
0,1,4,Who Made Who
1,2,1,Physical Graffiti
2,3,5,The dark side of the moon
3,4,3,Ride the Lightning
4,5,2,Selling England by the pound
5,6,7,The Queen Is Dead
6,7,6,Wish


For example, notice that (foreign key) *artist_id* for "Who made who" album is $4$ and this is equal to 
(the primary key) *id* in the artist table for AC/DC.

#### Populate the track table

In this table, we have two foreign keys, *album_id* and *genre_id*. 

In [16]:
cur.execute("""INSERT INTO track (title,album_id,genre_id,len,rating,repr) VALUES 
    ('Time',3,1,401,5,27),
    ('Firth of Fifth',5,1,521,5,18),
    ('The Rover',2,1,498,5,22),
    ('For Whom The Bell Tolls',4,1,390,4,11),
    ('You Shook Me All Night Long',1,1,330,5,15),
    ('The Boy with the Thorn in His Side',6,6,210,4,14),
    ('Friday I''m in Love',7,3,218,5,25)
    """)
db.commit()

In [17]:
# double check 
pd.read_sql_query("SELECT * FROM track", db)

,id,album_id,genre_id,title,len,rating,repr
0,1,3,1,Time,401,5,27
1,2,5,1,Firth of Fifth,521,5,18
2,3,2,1,The Rover,498,5,22
3,4,4,1,For Whom The Bell Tolls,390,4,11
4,5,1,1,You Shook Me All Night Long,330,5,15
5,6,6,6,The Boy with the Thorn in His Side,210,4,14
6,7,7,3,Friday I'm in Love,218,5,25


For example, notice that (the foreign key) *album_id* for "Time" song is $3$ and this is equal to 
(the primary key) *id* in the album table for "The dark side of the moon". 
Also (the foreign key) *genre_id* is $1$ and this is equal to (the primary key) *id* in the genre 
table for "Rock". 

## Querying the database (using JOIN)

Return **the name and genre** of all songs present in the database

In [18]:
pd.read_sql_query("SELECT track.title, genre.name FROM track JOIN genre ON track.genre_id = genre.id", db)

,title,name
0,Time,Rock
1,Firth of Fifth,Rock
2,The Rover,Rock
3,For Whom The Bell Tolls,Rock
4,You Shook Me All Night Long,Rock
5,The Boy with the Thorn in His Side,Alternative rock
6,Friday I'm in Love,Pop


Return **the tracks and the albums** of all songs present in the database

In [19]:
pd.read_sql_query("""SELECT track.title, album.title
    FROM track JOIN album JOIN genre
    ON track.genre_id = genre.id AND track.album_id = album.id
""",db)

,title,title
0,Time,The dark side of the moon
1,Firth of Fifth,Selling England by the pound
2,The Rover,Physical Graffiti
3,For Whom The Bell Tolls,Ride the Lightning
4,You Shook Me All Night Long,Who Made Who
5,The Boy with the Thorn in His Side,The Queen Is Dead
6,Friday I'm in Love,Wish


Return **the tracks, the artists, the albums, and the genres** of all songs

In [20]:
pd.read_sql_query("""SELECT track.title, artist.name, album.title, genre.name 
    FROM track JOIN artist JOIN album JOIN genre
    ON track.genre_id = genre.id 
    AND track.album_id = album.id 
    AND album.artist_id = artist.id
""", db)

,title,name,title,name
0,Time,Pink Floyd,The dark side of the moon,Rock
1,Firth of Fifth,Genesis,Selling England by the pound,Rock
2,The Rover,Led Zeppelin,Physical Graffiti,Rock
3,For Whom The Bell Tolls,Metallica,Ride the Lightning,Rock
4,You Shook Me All Night Long,AC/DC,Who Made Who,Rock
5,The Boy with the Thorn in His Side,The Smiths,The Queen Is Dead,Alternative rock
6,Friday I'm in Love,The Cure,Wish,Pop


#### Use AS

In [21]:
pd.read_sql_query("""SELECT t.title AS Song, art.name AS Artist, album.title AS Album, genre.name AS Genre
    FROM track AS t JOIN artist AS art JOIN album JOIN genre
    ON t.genre_id = genre.id 
    AND t.album_id = album.id 
    AND album.artist_id = art.id
""", db)

,Song,Artist,Album,Genre
0,Time,Pink Floyd,The dark side of the moon,Rock
1,Firth of Fifth,Genesis,Selling England by the pound,Rock
2,The Rover,Led Zeppelin,Physical Graffiti,Rock
3,For Whom The Bell Tolls,Metallica,Ride the Lightning,Rock
4,You Shook Me All Night Long,AC/DC,Who Made Who,Rock
5,The Boy with the Thorn in His Side,The Smiths,The Queen Is Dead,Alternative rock
6,Friday I'm in Love,The Cure,Wish,Pop


### Alter the table and insert new data

We want to be more precise about generes. For example, we see that "For Whom The Bell Tolls" is not properly 
rock but metal and "Firth of Fifth" is Progressive rock. 

We have to update the associated (foreign key) *genre_id* in the track table.

In [22]:
cur.execute("UPDATE track SET genre_id=2 WHERE title='For Whom The Bell Tolls'")
cur.execute("UPDATE track SET genre_id=4 WHERE title='Firth of Fifth'")
db.commit()

In [23]:
# show result
pd.read_sql_query("""SELECT track.title, artist.name, album.title, genre.name 
    FROM track JOIN artist JOIN album JOIN genre
    ON track.genre_id = genre.id 
    AND track.album_id = album.id 
    AND album.artist_id = artist.id
""", db)

,title,name,title,name
0,Time,Pink Floyd,The dark side of the moon,Rock
1,Firth of Fifth,Genesis,Selling England by the pound,Progressive rock
2,The Rover,Led Zeppelin,Physical Graffiti,Rock
3,For Whom The Bell Tolls,Metallica,Ride the Lightning,Metal
4,You Shook Me All Night Long,AC/DC,Who Made Who,Rock
5,The Boy with the Thorn in His Side,The Smiths,The Queen Is Dead,Alternative rock
6,Friday I'm in Love,The Cure,Wish,Pop


Now we want to add a new artist, one of its songs, the corresponding album, and the genre. In particular, we add a Michael Jackson's song. We need to add a new row in *artist* table, in *album* table, 
and *track* table, since the genre pop is already in.

In [24]:
cur.execute("INSERT INTO artist (name) VALUES ('Michael Jackson')")
db.commit()

In [25]:
pd.read_sql_query("SELECT * FROM artist",db)

,id,name
0,1,Led Zeppelin
1,2,Genesis
2,3,Metallica
3,4,AC/DC
4,5,Pink Floyd
5,6,The Cure
6,7,The Smiths
7,8,Michael Jackson


In [26]:
cur.execute("INSERT INTO album (title,artist_id) VALUES ('Thriller',8)")
db.commit()

In [27]:
pd.read_sql_query("SELECT * FROM album",db)

,id,artist_id,title
0,1,4,Who Made Who
1,2,1,Physical Graffiti
2,3,5,The dark side of the moon
3,4,3,Ride the Lightning
4,5,2,Selling England by the pound
5,6,7,The Queen Is Dead
6,7,6,Wish
7,8,8,Thriller


In [28]:
cur.execute("""INSERT INTO track (title,album_id,genre_id,len,rating,repr) VALUES ('Billie Jean',8,3,294,4,24)""")
db.commit()

In [29]:
pd.read_sql_query("SELECT * FROM track",db)

,id,album_id,genre_id,title,len,rating,repr
0,1,3,1,Time,401,5,27
1,2,5,4,Firth of Fifth,521,5,18
2,3,2,1,The Rover,498,5,22
3,4,4,2,For Whom The Bell Tolls,390,4,11
4,5,1,1,You Shook Me All Night Long,330,5,15
5,6,6,6,The Boy with the Thorn in His Side,210,4,14
6,7,7,3,Friday I'm in Love,218,5,25
7,8,8,3,Billie Jean,294,4,24


In [30]:
# Let us see show everything 
pd.read_sql_query("""SELECT t.title AS Song, art.name AS Artist, album.title AS Album, genre.name AS Genre
    FROM track AS t JOIN artist AS art JOIN album JOIN genre
    ON t.genre_id = genre.id 
    AND t.album_id = album.id 
    AND album.artist_id = art.id
""", db)

,Song,Artist,Album,Genre
0,Time,Pink Floyd,The dark side of the moon,Rock
1,Firth of Fifth,Genesis,Selling England by the pound,Progressive rock
2,The Rover,Led Zeppelin,Physical Graffiti,Rock
3,For Whom The Bell Tolls,Metallica,Ride the Lightning,Metal
4,You Shook Me All Night Long,AC/DC,Who Made Who,Rock
5,The Boy with the Thorn in His Side,The Smiths,The Queen Is Dead,Alternative rock
6,Friday I'm in Love,The Cure,Wish,Pop
7,Billie Jean,Michael Jackson,Thriller,Pop


## Further queries

#### Find songs by genre

In [31]:
# Find all Rock songs
pd.read_sql_query("""SELECT 
    track.title AS Song, 
    artist.name AS Artist, 
    album.title AS Album, 
    genre.name AS Genre,
    track.rating AS Rating,
    track.repr AS Reproductions
    FROM track JOIN artist JOIN album JOIN genre
    ON track.genre_id = genre.id AND track.album_id = album.id AND album.artist_id = artist.id
    WHERE genre.name = 'Rock' 
""", db)

,Song,Artist,Album,Genre,Rating,Reproductions
0,Time,Pink Floyd,The dark side of the moon,Rock,5,27
1,The Rover,Led Zeppelin,Physical Graffiti,Rock,5,22
2,You Shook Me All Night Long,AC/DC,Who Made Who,Rock,5,15


#### Find songs by artist

In [32]:
#Find all Michael Jackson's songs
pd.read_sql_query("""SELECT 
    track.title AS Song, 
    artist.name AS Artist, 
    album.title AS Album, 
    genre.name AS Genre,
    track.rating AS Rating,
    track.repr AS Reproductions
    FROM track JOIN artist JOIN album JOIN genre
    ON track.genre_id = genre.id AND track.album_id = album.id AND album.artist_id = artist.id
    WHERE Artist = 'Michael Jackson' 
""", db)

,Song,Artist,Album,Genre,Rating,Reproductions
0,Billie Jean,Michael Jackson,Thriller,Pop,4,24


#### Find songs of different genres

In [33]:
# Find all Pop *AND* Metal songs 
pd.read_sql_query("""SELECT 
    track.title AS Song, 
    artist.name AS Artist, 
    album.title AS Album, 
    genre.name AS Genre,
    track.rating AS Rating,
    track.repr AS Reproductions
    FROM track JOIN artist JOIN album JOIN genre
    ON track.genre_id = genre.id AND track.album_id = album.id AND album.artist_id = artist.id
    WHERE genre.name = 'Pop' OR genre.name = 'Metal' 
""", db)

,Song,Artist,Album,Genre,Rating,Reproductions
0,For Whom The Bell Tolls,Metallica,Ride the Lightning,Metal,4,11
1,Friday I'm in Love,The Cure,Wish,Pop,5,25
2,Billie Jean,Michael Jackson,Thriller,Pop,4,24


#### Constraints

In [34]:
# Find all Pop and Rock songs in the database whose reproductions are >= 23
pd.read_sql_query("""SELECT 
    track.title AS Song, 
    artist.name AS Artist, 
    album.title AS Album, 
    genre.name AS Genre,
    track.rating AS Rating,
    track.repr AS Reproductions
    FROM track JOIN artist JOIN album JOIN genre
    ON track.genre_id = genre.id AND track.album_id = album.id AND album.artist_id = artist.id 
        AND track.repr >= 23
    WHERE genre.name = 'Pop' OR genre.name = 'Rock' 
    """, db)

,Song,Artist,Album,Genre,Rating,Reproductions
0,Time,Pink Floyd,The dark side of the moon,Rock,5,27
1,Friday I'm in Love,The Cure,Wish,Pop,5,25
2,Billie Jean,Michael Jackson,Thriller,Pop,4,24


### Close the cursor and the connection

In [35]:
cur.close()
db.close()